In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Triton for Recommender Systems

The Triton Inference Server allows us to deploy our model to the web regardless of cloud provider, and it supports a number of different machine learning frameworks such as TensorFlow and PyTorch.

**Objectives:**

Learn how to deploy a model to Triton
1. Deploy saved NVTabular and PyTorch models to Triton Inference Server
2. Sent requests for predictions

**Pull and start Inference docker container**

At this point, before connecing to the Triton Server, we launch the inference docker container and then load the ensemble t4r_pytorch to the inference server. This is done with the scripts below:

launch the docker container:
```
docker run -it --gpus device=0 -p 8000:8000 -p 8001:8001 -p 8002:8002 -v <path_to_saved_models>:/root/models/ nvcr.io/nvidia/merlin/merlin-inference:0.6
```

This script will mount your local model-repository folder that includes your saved models from the previous cell to /root/models directory in the merlin-inference docker container.

start triton server:
After you started the merlin-inference container, you can start triton server with the command below. You need to provide correct path of the models folder.
```
tritonserver --model-repository=<path_to_models> --model-control-mode=explicit
```
Note: The model-repository path for our example is /root/models. The models haven't been loaded, yet. Below, we will request the Triton server to load the saved ensemble model below.

## 1. Deploy PyTorch and NVTabular Model to Triton Inference Server

Our Triton server has already been launched with to the web and is ready to make requests. We already, exportex the saved PyTorch model in the previous notebook, and generated the config files for Triton Inference Server.

In [2]:
 # Import dependencies
import os
from time import time

import argparse
import numpy as np
import pandas as pd
import sys
import cudf

## 1.2 Review exported files

Triton expects a specific directory structure for our models as the following format:

```
<model-name>/
[config.pbtxt]
<version-name>/
  [model.savedmodel]/
    <pytorch_saved_model_files>/
      ...
```

Let's check out our model repository layout. You can install tree library with apt-get install tree, and then run `!tree /workspace/models/` to print out the model repository layout as below:

Triton needs a [config file](https://github.com/triton-inference-server/server/blob/main/docs/model_configuration.md) to understand how to interpret the model. Let's look at the generated config file. It defines the input columns with datatype and dimensions and the output layer. Manually creating this config file can be complicated and NVTabular provides an easy function with `export_pytorch_ensemble` to deploy PyTorch model to Triton.


The config file needs the following information:
* [name](https://docs.nvidia.com/deeplearning/triton-inference-server/master-user-guide/docs/protobuf_api/model_config.proto.html#_CPPv4N6nvidia15inferenceserver11ModelConfig4nameE): The name of our model. Must be the same name as the parent folder.
* [platform](https://docs.nvidia.com/deeplearning/triton-inference-server/master-user-guide/docs/protobuf_api/model_config.proto.html#_CPPv4N6nvidia15inferenceserver11ModelConfig8platformE): The type of framework serving the model.
* [input](https://docs.nvidia.com/deeplearning/triton-inference-server/master-user-guide/docs/protobuf_api/model_config.proto.html#_CPPv4N6nvidia15inferenceserver11ModelConfig5inputE): The input our model expects.
  * `name`: Should correspond with the model input name.
  * `data_type`: Should correspond to the input's data type.
  * `dims`: The dimensions of the *request* for the input. For models that support input and output tensors with variable-size dimensions, those dimensions can be listed as -1 in the input and output configuration.
* [output](https://docs.nvidia.com/deeplearning/triton-inference-server/master-user-guide/docs/protobuf_api/model_config.proto.html#_CPPv4N6nvidia15inferenceserver11ModelConfig6outputE): The output parameters of our model.
  * `name`: Should correspond with the model output name.
  * `data_type`: Should correspond to the output's data type.
  * `dims`: The dimensions of the output.

## 1.3. Loading Model

Next, let's build a client to connect to our server. This InferenceServerClient object is what we'll be using to talk to Triton.

In [3]:
import tritonhttpclient

try:
    triton_client = tritonhttpclient.InferenceServerClient(url="localhost:8000", verbose=True)
    print("client created.")
except Exception as e:
    print("channel creation failed: " + str(e))
triton_client.is_server_live()

client created.
GET /v2/health/live, headers None
<HTTPSocketPoolResponse status=200 headers={'content-length': '0', 'content-type': 'text/plain'}>


/opt/conda/lib/python3.8/site-packages/tritonhttpclient/__init__.py:31: DeprecationWarning: The package `tritonhttpclient` is deprecated and will be removed in a future version. Please use instead `tritonclient.http`
  warnings.warn(


True

In [4]:
triton_client.get_model_repository_index()

POST /v2/repository/index, headers None

<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '245'}>
bytearray(b'[{"name":"t4r_pytorch","version":"1","state":"UNAVAILABLE","reason":"unloaded"},{"name":"t4r_pytorch_nvt","version":"1","state":"UNAVAILABLE","reason":"unloaded"},{"name":"t4r_pytorch_pt","version":"1","state":"UNAVAILABLE","reason":"unloaded"}]')


[{'name': 't4r_pytorch',
  'version': '1',
  'state': 'UNAVAILABLE',
  'reason': 'unloaded'},
 {'name': 't4r_pytorch_nvt',
  'version': '1',
  'state': 'UNAVAILABLE',
  'reason': 'unloaded'},
 {'name': 't4r_pytorch_pt',
  'version': '1',
  'state': 'UNAVAILABLE',
  'reason': 'unloaded'}]

- We load the ensemble model

In [5]:
model_name = "t4r_pytorch"
triton_client.load_model(model_name=model_name)

POST /v2/repository/models/t4r_pytorch/load, headers None

<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '0'}>
Loaded model 't4r_pytorch'


If all models are loaded succesfully, you should be seeing successfully loaded status next to each model name on your terminal.

## 2. Sent Requests for Predictions

- Load raw data for inference: We select the first 50 interactions and filter out sessions with less than 2 interactions

In [6]:
import cudf
batch = cudf.read_parquet('/workspace/data/Oct-2019.parquet').iloc[:100,:]

In [7]:
sessions_to_use = batch.user_session.value_counts()[batch.user_session.value_counts() > 1].index.values
filtered_batch = batch[batch.user_session.isin(sessions_to_use)]

In [8]:
filtered_batch.head()

,user_session,event_type,product_id,category_id,category_code,brand,price,user_id,event_time_ts,prod_first_event_time_ts
0,43,view,5300797,2053013563173241677,<NA>,panasonic,39.90,513903572,1570460611,1569948287
1,43,view,5300798,2053013563173241677,<NA>,panasonic,32.18,513903572,1570460616,1569934097
2,43,view,5300284,2053013563173241677,<NA>,rowenta,30.86,513903572,1570460621,1569927253
3,43,view,5300382,2053013563173241677,<NA>,remington,28.22,513903572,1570460636,1570026747
4,43,view,5300366,2053013563173241677,<NA>,polaris,26.46,513903572,1570460650,1570097085


In [9]:
import warnings

warnings.filterwarnings("ignore")

In [10]:
import nvtabular.inference.triton as nvt_triton
import tritonclient.grpc as grpcclient

inputs = nvt_triton.convert_df_to_triton_input(filtered_batch.columns, filtered_batch, grpcclient.InferInput)

output_names = ["output"]

outputs = []
for col in output_names:
    outputs.append(grpcclient.InferRequestedOutput(col))
    
MODEL_NAME_NVT = "t4r_pytorch"

with grpcclient.InferenceServerClient("localhost:8001") as client:
    response = client.infer(MODEL_NAME_NVT, inputs)
    print(col, ':\n', response.as_numpy(col))

output :
 [[-14.327803 -14.31179   -8.603794 ... -15.728743 -15.332325 -15.000166]
 [-17.512676 -16.658257  -7.375148 ... -18.469122 -17.989824 -17.459372]
 [-23.414373 -24.599703  -5.441975 ... -26.738302 -26.056658 -25.856808]
 [-14.12752  -13.941931  -8.50183  ... -14.946888 -14.655097 -14.412033]]


- Visualise top-k predictions

In [11]:
from demo_utils import visualize_response
visualize_response(filtered_batch, response, top_k=5)

- Top-5 predictions for session `17`: 604 || 4 || 11 || 560 || 619

- Top-5 predictions for session `43`: 270 || 4 || 3 || 116 || 349

- Top-5 predictions for session `217`: 173 || 908 || 349 || 881 || 164

- Top-5 predictions for session `317`: 27 || 11 || 2 || 3 || 4



- Unload models

In [12]:
triton_client.unload_model(model_name="t4r_pytorch")
triton_client.unload_model(model_name="t4r_pytorch_nvt")
triton_client.unload_model(model_name="t4r_pytorch_pt")

POST /v2/repository/models/t4r_pytorch/unload, headers None
{"parameters":{"unload_dependents":false}}
<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '0'}>
Loaded model 't4r_pytorch'
POST /v2/repository/models/t4r_pytorch_nvt/unload, headers None
{"parameters":{"unload_dependents":false}}
<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '0'}>
Loaded model 't4r_pytorch_nvt'
POST /v2/repository/models/t4r_pytorch_pt/unload, headers None
{"parameters":{"unload_dependents":false}}
<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '0'}>
Loaded model 't4r_pytorch_pt'
